In [4]:
!pip install tensorflow===2.2.0

ERROR: Could not find a version that satisfies the requirement tensorflow===2.2.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow===2.2.0


In [1]:
import tensorflow as tf


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import random

class DehazeModel:
    def __init__(self, model_path=None):
        """
        Initializes the DehazeModel class, loading the model if a model path is provided.
        
        Args:
        - model_path (str): Path to the saved model.
        """
        if model_path:
            self.load_model(model_path)
        else:
            self.model = None

    def load_model(self, model_path):
        """
        Loads the model architecture and weights.
        
        Args:
        - model_path (str): Path to the saved model file.
        """
        self.model = tf.keras.models.load_model(model_path)  # Load the entire model

    def preprocess_image(self, img_path):
        """
        Preprocesses an image from the specified path to prepare it for model input.
        
        Args:
        - img_path (str): Path to the image file.
        
        Returns:
        - img (tf.Tensor): Preprocessed image tensor ready for prediction.
        """
        # Load and decode the image
        img = tf.io.read_file(img_path)
        img = tf.io.decode_jpeg(img, channels=3)
        
        # Resize image to match the model's expected input size (224x224)
        img = tf.image.resize(img, size=(224, 224), antialias=True)
        img = img / 255.0  # Normalize to [0, 1] range
        img = tf.expand_dims(img, axis=0)  # Expand dims for batch size of 1
        return img
        
        # # Resize image based on its orientation
        # if img.shape[1] > img.shape[0]:  # Landscape orientation
        #     img = tf.image.resize(img, size=(1080, 1920), antialias=True)
        # elif img.shape[1] < img.shape[0]:  # Portrait orientation
        #     img = tf.image.resize(img, size=(1920, 1080), antialias=True)
        
        # img = img / 255.0  # Normalize to [0, 1] range
        # img = tf.expand_dims(img, axis=0)  # Expand dims for batch size of 1
        # return img

    def predict(self, img):
        """
        Predicts the dehazed image for the given preprocessed image tensor.
        
        Args:
        - img (tf.Tensor): Preprocessed image tensor.
        
        Returns:
        - tf.Tensor: Predicted dehazed image tensor.
        """
        if self.model is None:
            raise ValueError("Model not loaded. Please load the model first.")
        return self.model(img, training=False)

    def process_and_predict(self, img_path):
        """
        Processes an image and predicts the dehazed output.
        
        Args:
        - img_path (str): Path to the image file.
        
        Returns:
        - tf.Tensor: Predicted dehazed image tensor.
        """
        img = self.preprocess_image(img_path)
        return self.predict(img)

    def display_results(self, img_path, original_path):
        """
        Displays the hazy and predicted dehazed images side by side.
        
        Args:
        - img_path (str): Path to the hazy image.
        """
        hazy_img = self.preprocess_image(img_path)
        original_img = self.preprocess_image(original_path)
        dehazed_img = self.predict(hazy_img)

        # Display both the original hazy and dehazed images
        plt.figure(figsize=(10, 10))
        display_list = [hazy_img[0], dehazed_img[0], original_img[0]]
        title = ['Hazy Image', 'Dehazed Image','Original Image',]

        for i in range(3):
            plt.subplot(1, 3, i + 1)
            plt.title(title[i], fontsize=15)
            plt.imshow(display_list[i])
            plt.axis('off')
        
        plt.show()


In [1]:
import cv2
import numpy as np
import time
from threading import Thread

class RealTimeDehazer:
    def __init__(self, model_path, capture_interval=3, capture_index=0):
        """
        Initializes the RealTimeDehazer with the model and sets up camera.

        Args:
        - model_path (str): Path to the dehaze model.
        - capture_interval (int): Time in seconds between each dehazed frame capture.
        """
        # Load the dehazing model
        self.dehaze_model = DehazeModel(model_path)
        self.capture_interval = capture_interval  # Interval to capture image for dehazing
        self.last_capture_time = time.time()
        
        # Initialize the camera
        self.cap = cv2.VideoCapture(capture_index)  # Use 0 for default camera, or specify the external camera ID
        if not self.cap.isOpened():
            raise ValueError("Could not open video device")
        
        # Start a thread for real-time dehazing
        # self.hazify = self.add_haze
        # self.hazify = self.add_complex_haze
        self.hazify = self.add_uneven_haze
        self.haze_intensity = 0.4
        self.dehazed_image = None
        self.running = True
        self.dehaze_thread = Thread(target=self._dehaze_loop)
        self.dehaze_thread.start()
        

    def _dehaze_loop(self):
        """
        Continuously captures and dehazes images at the specified interval.
        """
        while self.running:
            current_time = time.time()
            # Capture and dehaze image every `capture_interval` seconds
            if current_time - self.last_capture_time >= self.capture_interval:
                ret, frame = self.cap.read()
                if ret:
                    # Preprocess and dehaze the captured frame
                    hazed_frame = self.hazify(frame, self.haze_intensity)
                    img_tensor = self.dehaze_model.preprocess_image_from_array(hazed_frame)
                    self.dehazed_image = self.dehaze_model.predict(img_tensor)[0].numpy()
                    self.last_capture_time = current_time
    
    # Preprocessing modification in DehazeModel to handle images from arrays
    def preprocess_image_from_array(self, img_array):
        """
        Preprocesses an image from an array for prediction.

        Args:
        - img_array (np.array): Image array captured from camera.

        Returns:
        - tf.Tensor: Preprocessed image tensor.
        """
        img = tf.image.resize(img_array, (224, 224))
        img = tf.convert_to_tensor(img, dtype=tf.float32) / 255.0  # Normalize to [0, 1]
        img = tf.expand_dims(img, axis=0)  # Add batch dimension
        return img

    def add_haze(self,image, haze_intensity=0.5):
        """
        Adds haze to an image.

        Args:
        - image (np.array): Original image.
        - haze_intensity (float): Intensity of the haze effect (0.0 to 1.0).

        Returns:
        - np.array: Hazy image.
        """
        # Create a solid haze layer (white color with some intensity)
        haze_layer = np.ones_like(image, dtype=np.float32) * 255  # White haze
        haze_layer = haze_layer.astype(np.uint8)

        # Blend the original image and the haze layer
        hazy_image = cv2.addWeighted(image, 1 - haze_intensity, haze_layer, haze_intensity, 0)
        return hazy_image
    
    def add_complex_haze(self, image, haze_intensity=0.5):
        """
        Adds a more complex haze to an image using random pixel values.

        Args:
        - image (np.array): Original image.
        - haze_intensity (float): Intensity of the haze effect (0.0 to 1.0).

        Returns:
        - np.array: Hazy image.
        """
        # Generate random haze layer
        haze_layer = np.random.randint(200, 255, size=image.shape, dtype=np.uint8)  # Random haze values

        # Blend the original image and the random haze layer
        hazy_image = cv2.addWeighted(image, 1 - haze_intensity, haze_layer, haze_intensity, 0)
        return hazy_image
    
    def add_uneven_haze(self, image, haze_intensity=0.8):
        """
        Adds uneven haze to an image based on a randomly generated mask.

        Args:
        - image (np.array): Original image.
        - haze_intensity (float): Intensity of the haze effect (0.0 to 1.0).

        Returns:
        - np.array: Hazy image.
        """
        # Generate random haze layer
        haze_layer = np.random.randint(200, 255, size=image.shape, dtype=np.uint8)

        # Create a mask for uneven haze (random noise)
        noise_mask = np.random.rand(*image.shape[:2])  # Random values between 0 and 1
        noise_mask = (noise_mask * 255).astype(np.uint8)  # Scale to [0, 255]

        # Apply a Gaussian blur to the noise mask for smoother transitions
        noise_mask = cv2.GaussianBlur(noise_mask, (21, 21), 0)

        # Normalize the mask to have values between 0 and 1
        normalized_mask = noise_mask.astype(np.float32) / 255.0

        # Compute beta as a single value (optional adjustment)
        beta = haze_intensity * np.mean(normalized_mask)  # Average to create a single float value for blending

        # Blend the original image and the haze layer unevenly based on the mask
        hazy_image = cv2.addWeighted(image, 1 - haze_intensity, haze_layer, beta, 0)
        return hazy_image

    def show_feed(self):
        """
        Displays the live camera feed and the dehazed image side by side.
        """
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to grab frame")
                break
            
            # Resize frame if needed
            frame = cv2.resize(frame, (640, 480))

            # hazy_frame = self.dehaze_model.preprocess_image_from_array(hazy_frame)
            # Add haze to the original frame
            # hazy_frame = self.add_haze(frame, haze_intensity=0.5)
            hazy_frame = self.hazify(frame, self.haze_intensity)

            # Prepare dehazed image for display
            if self.dehazed_image is not None:
                dehazed_display = (self.dehazed_image * 255).astype(np.uint8)
                dehazed_display = cv2.resize(dehazed_display, (640, 480))
            else:
                dehazed_display = np.zeros_like(frame)

            # Concatenate the hazy feed and dehazed images side by side
            combined_display = np.concatenate((hazy_frame, dehazed_display), axis=1)
            cv2.imshow("Camera Feed and Dehazed Image", combined_display)

            # Exit the display loop on pressing 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                self.stop()
                break


    def stop(self):
        """
        Stops the real-time dehazing and releases the camera.
        """
        self.running = False
        self.dehaze_thread.join()
        self.cap.release()
        cv2.destroyAllWindows()




In [2]:

# Usage:
# Create RealTimeDehazer instance and start real-time dehazing
realtime_dehazer = RealTimeDehazer(model_path='gman_net_model.h5')


NameError: name 'DehazeModel' is not defined

In [3]:
realtime_dehazer.show_feed()
# 

NameError: name 'realtime_dehazer' is not defined

In [28]:
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Function to calculate PSNR
def calculate_psnr(gt_image, pred_image):
    mse = np.mean((gt_image - pred_image) ** 2)
    if mse == 0:  # Avoid division by zero
        return float('inf')
    max_pixel = 255.0  # Assuming 8-bit image depth
    psnr = 10 * np.log10((max_pixel ** 2) / mse)
    return psnr

# Function to calculate SSIM
def calculate_ssim(gt_image, pred_image):
    ssim_value, _ = ssim(gt_image, pred_image, full=True, multichannel=True)
    return ssim_value

# Function to calculate MSE
def calculate_mse(gt_image, pred_image):
    mse = np.mean((gt_image - pred_image) ** 2)
    return mse

# Function to calculate Absolute Difference (AD)
def calculate_ad(gt_image, pred_image):
    ad = np.mean(np.abs(gt_image - pred_image))
    return ad

# Evaluation function for the model
def evaluate_dehaze_model(gt_images, pred_images):
    psnr_values = []
    ssim_values = []
    mse_values = []
    ad_values = []
    
    for gt_image, pred_image in zip(gt_images, pred_images):
        psnr_values.append(calculate_psnr(gt_image, pred_image))
        ssim_values.append(calculate_ssim(gt_image, pred_image))
        mse_values.append(calculate_mse(gt_image, pred_image))
        ad_values.append(calculate_ad(gt_image, pred_image))
    
    # Print the average results
    print("Average PSNR:", np.mean(psnr_values))
    print("Average SSIM:", np.mean(ssim_values))
    print("Average MSE:", np.mean(mse_values))
    print("Average AD:", np.mean(ad_values))



In [ ]:
# Example usage:
# Assuming you have lists `gt_images` and `pred_images` with image arrays
# You can read the images using cv2.imread or any other image loader
# e.g., gt_images = [cv2.imread("gt1.png"), cv2.imread("gt2.png"), ...]
#       pred_images = [cv2.imread("pred1.png"), cv2.imread("pred2.png"), ...]

# evaluate_dehaze_model(gt_images, pred_images)


